# Periscope Correction Current State

In 2010, 15 observations with both relatively bright sources and larger periscope gradient changes were used to generate a periscope gradient calibration.  This calibration was applied as a correction to the fiducial light positions directly.  See http://occweb.cfa.harvard.edu/twiki/Aspect/PeriscopeTilt  This correction is applied as part of current processing and was applied to historical observations as part of ReproIV efforts.

As mentioned, however, the correction was only calibrated with 15 observations with larger drifts, and current anecdotal observations 6 years later demonstrate uncorrected periscope drift effects.  So, to start, we want to gather new data and evaluate the "goodness" of the 2010 calibration against new data.  To do so we:

 - Get a new data set with both relatively on-axis bright point sources and large(r) expected periscope drifts
 - Evaluate X-ray event drift in Y and Z in this data set
 

## Get new data set

In [1]:
%pylab notebook
import os
from IPython.display import Image
from astropy.table import Table
import cPickle
from Ska.DBI import DBI

Populating the interactive namespace from numpy and matplotlib


In [2]:
workdir = '/data/aca/analysis/periscope_tilt_2016'

Run code that finds "tilted" observations, runs celldetect, gets the brightest source that is also pretty-much on-axis, and extracts events in a circular region around those sources.  The on-axis limit is set at a radial distance of 180 pixels.

In [3]:
sqlaca = DBI(dbi='sybase', server='sybase', user='aca_read')
tilted_obs = sqlaca.fetchall("""select obsid from obs_periscope_tilt
where max_oobagrd3 - min_oobagrd3 > 0.09 order by (max_oobagrd3 - min_oobagrd3) desc""")
just_long = sqlaca.fetchall("""select obsid from observations
where kalman_tstop - kalman_tstart > 80000 
and kalman_datestart > '2004'
order by kalman_tstart""")

In [ ]:
len(just_long)

566

In [ ]:
import tilt.find_sources
tilt.find_sources.get_xray_data(just_long['obsid'])

Remaking /proj/sot/ska/analysis/periscope_tilt_2016/auto/obs09148/point_source.fits
making released_pos.pkl for 9148


{'dec_pnt': -0.01468732104276346, 'ccds4_on': 'Y', 'roll_pnt': 312.1566290001749, 'ccdi1_on': 'N', 'readmode': 'TIMED', 'datamode': 'FAINT', 'kalman_done': 1, 'date_end': '2008-12-06T07:17:53', 'dither_y_freq': None, 'z_det_offset': -0.25, 'ra_pnt': 40.67423912788944, 'sched_exp_time': 80700.0, 'obsid': 9148, 'si_mode': 'TE_0036C', 'obi_num': 0, 'obsid_datestart': '2008:340:08:04:55.300', 'ccds1_on': 'Y', 'num_ccd_on': 6, 'obs_mode': 'POINTING', 'dither_z_phase': None, 'dec_targ': -0.013333, 'sim_z_offset': -3.349067460718516, 'ra_nom': 40.67423912788944, 'ccds0_on': 'Y', 'defocus': 0.001444936568705701, 'sim_z': -186.7834551222893, 'kalman_tstart': 344853869.83386177, 'obs_id': 9148, 'instrume': 'ACIS', 'tstart': 344852685.9588, 'grating': 'HETG', 'obsid_tstart': 344851559.4837477, 'detector': 'ACIS-S', 'ascdsver': '7.6.11.9', 'kalman_tstop': 344934755.66288537, 'sim_y': 0.0, 'revision': 1, 'dec_nom': -0.01468732104276346, 'kalman_datestart': '2008:340:08:43:25.650', 'dither_y_phase'

Run one script that makes ds9 preview images of the sources, and another that quickly lets one manually review those images.  review_images is set to only review sources that show up with 2500 counts or more.

In [ ]:
# run tilt/make_check_images.py
# run tilt/review_images.py

For each source, we have several useful files in the obsid directories.  

- picked_src.dat information from celldetect about the picked/selected source
- point_source.fits dmcopy/extracted region of events from evt2 file from the observation
- point_stat.dat text file with True or False indicating point-source-ness from manual review
- released_pos.pkl Y,Z positions of events in the circular region extracted around the source

In [ ]:
!ls auto/obs17128

And a master file with the sources

In [ ]:
srcs = Table.read('src_table.dat', format='ascii')
print srcs[['obsid','NET_COUNTS','point_source']][:1]

## Evaluate 

In [ ]:
# this would probably be nicer with scipy.stats.binned_statistic, but that doesn't
# give me nice error bars, and this was hanging around
def binned_mean(data, evtstime, tbin=10000.):
    ts = []
    ds = []
    dsminus = []
    dsplus = []
    for win_start in range(0,
                           int(evtstime[-1]-evtstime[0])-int(tbin),
                           int(tbin)):
        tmask = ((evtstime-evtstime[0] >= win_start)
                 & (evtstime-evtstime[0] < win_start + int(tbin)))
        range_data = data[tmask]
        range_time = evtstime[tmask]
        if np.std(range_data) > 0:
            ds.append(np.mean(range_data))
            dsminus.append(np.std(range_data)/np.sqrt(len(range_data)))
            dsplus.append(np.std(range_data)/np.sqrt(len(range_data)))
            t = np.mean(range_time)
            ts.append(t)
    return [ts, ds, dsminus, dsplus]

First, here's a demo image of one of these 'pointy' sources:

In [ ]:
obsid = 17128
Image("auto/obs{:05d}/ds9_src.png".format(obsid))

And if we plot up the Y-angle position of the events binned by time there does appear to be drift.

In [ ]:
obsid = 17128
pos = cPickle.load(open('auto/obs{:05d}/released_pos.pkl'.format(obsid)))
ts, ds, dsminus, dsplus = binned_mean(pos['yag'], pos['time'], 10000)
errorbar(ts, ds, yerr=dsminus, linestyle='');
ylabel('event y-angle (arcsec)')
margins(x=.1)
title('obsid {} y-angle of src (5ks bins)'.format(obsid));

We basically care about this in aggregate for this whole source set, about each sources mean position.  First, just this source:

In [ ]:
hist(ds - np.mean(ds), bins=arange(-.5, .5, .05), log=True);

In [ ]:
# then all of the sources we've marked as point sources thus far with NET_COUNTS > 2500
binsize = 10000
bin_residuals = []
bin_obsids = []
obsids = []
per_obs_p2p = []
per_obs_std = []
ax = 'yag'
for obsid in srcs[(srcs['NET_COUNTS'] > 2500) & (srcs['point_source'] == 'True')]['obsid']:
    pkl = 'auto/obs{:05d}/released_pos.pkl'.format(obsid)
    if not os.path.exists(pkl):
        continue
    pos = cPickle.load(open(pkl))
    ts, ds, dsminus, dsplus = binned_mean(pos[ax], pos['time'], binsize)
    bin_residuals.extend((ds - np.mean(ds)).tolist())
    bin_obsids.extend(np.repeat(obsid, len(ds)))
    obsids.append(obsid)
    per_obs_p2p.append(np.max(ds - np.mean(ds)) - np.min(ds - np.mean(ds)))
    per_obs_std.append(np.std(ds - np.mean(ds)))
bin_obsids = np.array(bin_obsids)
bin_residuals = np.array(bin_residuals)

In [ ]:
len(obsids)

In [ ]:
step = 0.025
hist(bin_residuals, bins=arange(np.min(bin_residuals), np.max(bin_residuals) + step, step), log=True);

In [ ]:
hist(per_obs_p2p, bins=arange(0, .36, .025));

In [ ]:
hist(per_obs_p2p, bins=arange(0, .36, .025), cumulative=True, normed=True);
grid();

In [ ]:
last_cal_observations = [10062, 10227, 10228, 11075, 11260, 11688, 11823, 12088, 5003,
                         7077, 7079, 8922, 9133, 9218, 9407, 9532]

In [ ]:
from Ska.DBI import DBI
sqlaca = DBI(dbi='sybase', server='sybase', user='aca_read')
old_gradients = sqlaca.fetchall("""select obsid, max_oobagrd3 - min_oobagrd3 as grad_diff
from obs_periscope_tilt
where obsid in ({})""".format(",".join([str(o) for o in last_cal_observations])))

In [ ]:
hist(old_gradients['grad_diff']);

In [ ]:
cal_srcs = srcs[(srcs['NET_COUNTS'] > 2500) & (srcs['point_source'] == 'True')]
all_gradients = sqlaca.fetchall("""select max_oobagrd3 - min_oobagrd3 as grad_diff
from obs_periscope_tilt
where obsid in ({})""".format(",".join([str(o) for o in cal_srcs['obsid']])))

In [ ]:
hist(all_gradients['grad_diff'], log=True);

So, we confirm that there are both uncorrected offsets and a much larger range of periscope drifts available.

## Setup for new fit

Because all of the X-ray events and aspect solutions are already corrected, we can either fit for an additive correction on the current model, or we can back out the current correction and fit new terms on "raw" data.  I have preferred the second.  There are actually two ways to get at the "raw" data.  The strict method would be to rerun the aspect pipeline on all observations with the periscope correction disabled, and then repeat this source extraction.  A more time-efficient method is to reverse the correction on the X-ray event Y and Z positions directly, which is what I have done.  I have, however, confirmed that the positions via this direct method are very very close to what you get if you run acis_process_events on these events using an aspect solution with the periscope correction disabled.  Let's walk through that.

In [ ]:
import os
from Ska.File import chdir
from Ska.Shell import bash

Run the current aspect pipeline with and without the periscope correction

In [ ]:
obsid = 11688.
obsdir = 'runpipeline/obs{:05}'.format(obsid)
if not os.path.exists(obsdir):
    os.makedirs(obsdir)
    with chdir(obsdir):
        bash("python {}/runasp.py --obsid {} --param 'apply_pericorr=no' --label uncorr".format(workdir, obsid))
        bash("python {}/runasp.py --obsid {} --label corr".format(workdir, obsid))

Process the point source events with each aspect solution

```
cd runpipeline
# get other acis level 1 pieces
arc5gl
obsid=11688
get acis1
quit
# re-extract the point source and replace the event file with the point source file
cp ../auto/obs11688/center.reg .
# though I just used the text from center.reg in my dmcopy
ciao
dmcopy acisf11688_001N002_evt1.fits'[cols  time,ra,dec,x,y,ccd_id,node_id,expno,chip,tdet,det,sky,phas,pha,pha_ro,energy,pi,fltgrade,grade, \
status][(x,y)=circle(4059.416856, 4091.439636, 6)]' point_evt1.fits
mv acisf11688_001N002_evt1.fits.gz old_event_file.fits.gz
# link the uncorrected aspect solution
ln -s obs11688/pipeline_out/uncorr/out1/*asol* pcad_asol1.fits
# run the ciao commands to process events 
sh ../my_repro.sh
mv acis_dstrk_evt2.fits acis_uncorr_evt2.fits
# link the corrected aspect solution
rm pcad_asol1.fits
ln -s obs11688/pipeline_out/corr/out1/*asol* pcad_asol1.fits
sh ../my_repro.sh
mv acis_dstrk_evt2.fits acis_corr_evt2.fits
# make versions with computed ra dec in the files instead of as transforms
dmcopy acis_uncorr_evt2.fits'[cols time,ra,dec,x,y]' acis_uncorr_radec_evt2.fits
dmcopy acis_corr_evt2.fits'[cols time,ra,dec,x,y]' acis_corr_radec_evt2.fits
```

In [ ]:
from astropy.table import Table
from astropy.io import fits
from Ska.quatutil import radec2yagzag
from Quaternion import Quat
from Ska.Matplotlib import plot_cxctime
from Ska.engarchive import fetch
from Ska.Numpy import interpolate, smooth

Read in the events from the corrected and uncorrected sapect solutions

In [ ]:
uncorr = Table.read('runpipeline/acis_uncorr_radec_evt2.fits')
uy, uz = radec2yagzag(uncorr['RA'], uncorr['DEC'], q)

In [ ]:
corr = Table.read('runpipeline/acis_corr_radec_evt2.fits')

In [ ]:
def filter_bad_telem(msid, method='nearest'):
    # use the bad quality field to select
    # and replace bad data in place using the given method
    ok = msid.bads == False
    bad = msid.bads == True
    fix_vals = interpolate(msid.vals[ok],
                           msid.times[ok],
                           msid.times[bad],
                           method=method)
    msid.vals[bad] = fix_vals
    return msid

Take a corrected file, reverse the correction using the 2010 gradient values, and return "uncorrected" y and z.

In [ ]:
def reverse_periscope_corr(corr_file):
    evts = Table.read(corr_file)
    f_corr = fits.open(corr_file)
    q = Quat((f_corr[1].header['RA_NOM'], f_corr[1].header['DEC_NOM'], f_corr[1].header['ROLL_NOM']))
    y, z = radec2yagzag(corr['RA'], corr['DEC'], q)
    
    GRADIENTS = dict(OOBAGRD3=dict(yag=6.98145650e-04,
                                zag=9.51578351e-05,
                                ),
                OOBAGRD6=dict(yag=-1.67009240e-03,
                            zag=-2.79084775e-03,
                            ))
    tstart = evts['time'][0]
    tstop = evts['time'][-1]
    gradients = fetch.MSIDset(GRADIENTS.keys(), tstart-100, tstop+100)
    y, z = radec2yagzag(evts['RA'], evts['DEC'], q)
    for msid in gradients:
        # filter bad telemetry in place
        filter_bad_telem(gradients[msid])
        times = gradients[msid].times
        evt_idx = np.searchsorted(times, evts['time'])
        # find a mean gradient, because this calibration is relative to mean
        mean_gradient = np.mean(gradients[msid].vals[evt_idx])
        # and smooth the telemetry to deal with slow changes and large step sizes..
        smooth_gradient = smooth(gradients[msid].vals)
        y += (smooth_gradient[evt_idx] - mean_gradient) * GRADIENTS[msid]['yag']
        z += (smooth_gradient[evt_idx] - mean_gradient) * GRADIENTS[msid]['zag']
        
    return evts['time'], y, z

In [ ]:
etime, y, z = reverse_periscope_corr('runpipeline/acis_corr_radec_evt2.fits')

In [ ]:
print np.max(np.abs(y -uy)*3600)
figure();
hist((y - uy) * 3600, bins=np.arange(-.025,.025, .0001));
xlabel('y diff arcsec');

In [ ]:
figure()
print np.max(np.abs(z - uz)*3600)
hist((z - uz) * 3600, bins=np.arange(-.025,.025, .0001));
xlabel('z diff arcsec');

The differences between the uncorrected values and the corrected values with the inverse correction applied are negligible.